In [88]:
from yaml import load, dump
import json

JOB_NAME = "cypress_run"
INPUT_FILE = "github/" + JOB_NAME + ".yml"
OUTPUT_FILE = "Outputs/github/" + JOB_NAME + ".json"

try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper
    

In [90]:
yaml_file = open(INPUT_FILE, 'r')
yaml_content = load(yaml_file, Loader=Loader)
with open(OUTPUT_FILE, "w") as outfile:
    json.dump(yaml_content, outfile)
# print("Key: Value")
# for key, value in yaml_content.items():
#     print(f"{key}: {value}")

In [43]:
def browseDict(keyElt, dictionary):
    NOT_FOUND = ""
    for key, value in dictionary.items():        
        if (key == keyElt):
            return (keyElt, value)
        if (isinstance(value,dict)):
            result = browseDict(keyElt, value)
            if(result != NOT_FOUND):
                return result
    return NOT_FOUND           

In [91]:
def get_variables(dictionary):
    variables = {}
    _, value = browseDict("workflow_call", dictionary)
    inputs = value.get("inputs", "")
    secrets = value.get("secrets", "")
    if(inputs):
        for key, value in inputs.items():
            variables[key] = value.get("default", "")
    if(secrets):
        for key, value in secrets.items():
            variables[key] = value.get("default", "")
    return variables

#get_variables(yaml_content)

In [84]:
def get_docker_image(job):
    container = job.get("container", "")
    if(container):
        if(isinstance(container, str)):
            return container
        elif(isinstance(container, dict)):
            return container.get("image", "")
    return ""
    
def get_runs_on(job):
    return job.get("runs-on", "")

def get_steps(job):
    return job.get("steps", [])

def get_artifacts(steps):
    artifacts = {}
    for elt in steps:
        if "actions/upload-artifact" in elt.get("uses", ""):
            artifacts["name"] = elt["with"].get("name", "")
            artifacts["paths"] = elt["with"].get("path", "").splitlines()
    return artifacts
    
def get_cache(steps):
    cache = {}
    for elt in steps:
        if "actions/cache" in elt.get("uses", ""):
            cache["key"] = elt["with"].get("key", "")
            cache["paths"] = elt["with"].get("path", "").splitlines()
    return cache
     

In [92]:
jobs_list = []
_,jobs = browseDict("jobs", yaml_content)
for key, value in jobs.items():
    jobs_list.append((key, value))
_, job = jobs_list[0]

In [93]:
steps = get_steps(job)
artifacts = get_artifacts(steps)
cache = get_cache(steps)
print("Artifacts:",artifacts)
print("Cache", cache)

Artifacts: {'name': 'cypress-screenshots', 'paths': ['cypress/screenshots']}
Cache {'key': "${{runner.os}}-build-${{env.cache-name}}-${{hashFiles('**/package-lock.json')}}", 'paths': ['~/.npm', '~/.cache/Cypress']}


In [81]:
lines = "~/.npm\n~/.cache/Cypress\n".splitlines()
for line in lines:
    print(line)
print("Lines", lines)

Lines []


In [ ]:
from os.path import basename, splitext
from collections import OrderedDict
gitlab_cicd_dict = OrderedDict()
job_name,_ = splitext(basename(INPUT_FILE))
gitlab_cicd_dict[job_name] = {
    "image": "",
    "cache": {},
    "variables": {},
    "script": []
}
            
